In [1]:
import os
import numpy as np
from tqdm import tqdm
from multiprocessing import Pool, cpu_count

import torch
import torch.nn as nn
import torch.nn.functional as F

import neurox.interpretation.utils as utils
import neurox.interpretation.ablation as ablation
import neurox.interpretation.linear_probe as linear_probe

from ttsxai.utils.utils import read_ljs_metadata


log_dir = '/nas/users/dahye/kw/tts/ttsxai/logs/probe_tacotron2_duration'
data_activation_dir = "/nas/users/dahye/kw/tts/ttsxai/data_activation/LJSpeech/tacotron2_waveglow"


In [2]:
# from ttsxai.utils import jupyter_utils
# jupyter_utils.register_always_reload(packages=[
#     'ttsxai.interfaces',
#     # 'ttsxai.models.text2mel',
#     # 'ttsxai.models.mel2wave',
#     # 'ttsxai.utils',
# ])
# jupyter_utils.register_always_reload(modules=[
#     # 'ttsxai.text',
#     # 'ttsxai.models.text2mel.tacotron2',
#     # 'ttsxai.models.text2mel.fastspeech2',
#     # 'ttsxai.utils.plot_utils',
#     # 'ttsxai.pitch_calculator',
#     # 'ttsxai.energy_calculator'
# ])

In [2]:
import IPython.display as ipd
from matplotlib import pyplot as plt

import torch

from ttsxai.utils.utils import read_ljs_metadata
from ttsxai.utils import plot_utils
from ttsxai.interfaces.tts_interface import TTSInterface, get_text2mel, get_mel2wave

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

data_dir = '/nas/users/dahye/kw/tts/ttsxai/data/LJSpeech/metadata.csv'
# text_dict = read_ljs_metafile(data_dir)
text_dict = read_ljs_metadata(mode='train')

text2mel = get_text2mel('tacotron2', device)
mel2wave = get_mel2wave('waveglow', device)

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



/nas/users/dahye/kw/tts/ttsxai/src/waveglow/glow.py:74: UserWarning: torch.qr is deprecated in favor of torch.linalg.qr and will be removed in a future PyTorch release.
The boolean parameter 'some' has been replaced with a string parameter 'mode'.
Q, R = torch.qr(A, some)
should be replaced with
Q, R = torch.linalg.qr(A, 'reduced' if some else 'complete') (Triggered internally at ../aten/src/ATen/native/BatchLinearAlgebra.cpp:2349.)
  W = torch.qr(torch.FloatTensor(c, c).normal_())[0]
/nas/users/dahye/kw/tts/ttsxai/src/tacotron2/stft.py:67: FutureWarning: Pass size=1024 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  fft_window = pad_center(fft_window, filter_length)


In [162]:
x[:, -512:].shape

torch.Size([27, 512])

In [452]:
text2mel = get_text2mel('tacotron2', device)

# 예시: encoder의 특정 레이어의 활성화 값을 사전에 지정된 값으로 변경
# modified_activations = {
#     # 'lstm': torch.randn(1, 27, 512).cuda(),  # 임의의 값으로 설정
#     # 'lstm': torch.from_numpy(output_dict['activations']['lstm'][None, :]).cuda(),
#     'lstm': x[:, -512:][None, :].cuda(),
#     # 다른 레이어에 대한 변경 사항 추가 가능
# }

modified_activations = {
    # 'lstm': torch.randn(1, 27, 512).cuda(),  # 임의의 값으로 설정
    # 'lstm': torch.from_numpy(0.1 + output_dict['activations']['lstm'][None, :]).cuda(),
    # 'conv_0': x[:, :512][None, :].cuda(),
    # 'conv_1': x[:, 512:1024][None, :].cuda(),
    # 'conv_2': x[:, 1024:1536][None, :].cuda(),
    # 'lstm': x[:, 1536:2048][None, :].cuda(),
    'lstm': x[None, :].cuda(),
    # 'lstm': torch.from_numpy(X_test[:27, -512:])[None, :].cuda(),
    # 다른 레이어에 대한 변경 사항 추가 가능
}

text2mel.set_modified_activations(modified_activations)

In [313]:
nid2name = get_neuronidx2name(output_dict['activations'])

In [314]:
new_dict = output_dict['activations'].copy

In [321]:
x.shape

torch.Size([27, 51])

In [361]:
new_dict = output_dict['activations'].copy()

In [367]:
new_dict['lstm'].shape

torch.Size([1, 27, 512])

In [402]:
jupyter_utils.unregister_always_reload()

In [468]:
new_dict = output_dict['activations'].copy()

for oi, o in enumerate(ordering[:n]):
    name, i = nid2name[o].split('__')
    new_dict[name][:, int(i)] = x[:, oi].cpu().detach().numpy()
    print(name, i)

for k, v in new_dict.items():
    new_dict[k] = torch.tensor(v).cuda()[None, :] 

text2mel.set_modified_activations(new_dict)

conv_2 380
lstm 447
lstm 494
lstm 39
lstm 328
conv_2 78
lstm 197
conv_2 444
lstm 323
lstm 26
lstm 165
lstm 463
lstm 103
conv_1 361
lstm 92
lstm 19
lstm 144
lstm 446
conv_2 204
conv_2 289
lstm 326
conv_2 503
lstm 169
lstm 178
lstm 24
lstm 65
lstm 327
conv_2 283
lstm 106
conv_2 401
lstm 155
lstm 29
lstm 225
lstm 89
lstm 305
lstm 34
lstm 148
conv_2 315
lstm 208
conv_2 294
conv_2 321
lstm 91
conv_2 171
lstm 331
lstm 245
lstm 52
conv_1 310
lstm 486
lstm 390
lstm 386
conv_2 433


In [458]:
x.shape

torch.Size([27, 51])

In [443]:
text2mel.set_modified_activations({})

In [3]:
tts = TTSInterface(
    device=device,
    text2mel=text2mel,
    mel2wave=mel2wave 
).to(device)

text = text_dict['LJ001-0002']
output_dict = tts.forward(text)
wave = output_dict['wave']

ipd.Audio(wave, rate=tts.sampling_rate)

(80, 171)


/nas/users/dahye/kw/tts/ttsxai/src/tacotron2/audio_processing.py:50: FutureWarning: Pass size=1024 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  win_sq = librosa_util.pad_center(win_sq, n_fft)


In [7]:
output_dict.keys()

dict_keys(['wave', 'mel', 'text', 'phone', 'phonesymbols', 'token', 'duration', 'alignment', 'activations'])

In [9]:
output_dict['phonesymbols']

['IH0',
 'N',
 ' ',
 'B',
 'IY1',
 'IH0',
 'NG',
 ' ',
 'K',
 'AH0',
 'M',
 'P',
 'AE1',
 'R',
 'AH0',
 'T',
 'IH0',
 'V',
 'L',
 'IY2',
 ' ',
 'M',
 'AA1',
 'D',
 'ER0',
 'N',
 '.']

In [121]:
output_dict['activations']['lstm'][None, :].shape

(1, 27, 512)

In [118]:
activations2dict(activations2array(output_dict['activations']))

{'conv_0': array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.05071047],
        [0.        , 0.        , 0.        , ..., 0.17463359, 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.08041617,
         0.        ],
        ...,
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.03786207],
        [0.        , 0.        , 0.        , ..., 0.40203983, 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ]], dtype=float32),
 'conv_1': array([[0.       , 0.       , 0.       , ..., 0.       , 0.       ,
         0.4442848],
        [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
         0.       ],
        [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
         0.       ],
        ...,
        [0.0410802, 0.       , 0.       , ..., 0.       , 0.       ,
         0.       ],
 

In [109]:
type(activations2array(output_dict['activations']))

numpy.ndarray

In [106]:
type(output_dict['activations']) == dict

True

In [113]:
output_dict['activations']['conv_0'].shape

(27, 512)

In [102]:
def get_neuronidx2name(d):
    mapping = {}
    current_start_index = 0
    for layer_name, activations in d.items():
        # Calculate the end index for this activation
        end_index = current_start_index + activations.shape[1] - 1
        for i in range(current_start_index, end_index + 1):
            mapping[i] = f'{layer_name}__{i - current_start_index}'
        current_start_index = end_index + 1
    return mapping

In [103]:
get_neuronidx2name(output_dict['activations'])

{0: 'conv_0__0',
 1: 'conv_0__1',
 2: 'conv_0__2',
 3: 'conv_0__3',
 4: 'conv_0__4',
 5: 'conv_0__5',
 6: 'conv_0__6',
 7: 'conv_0__7',
 8: 'conv_0__8',
 9: 'conv_0__9',
 10: 'conv_0__10',
 11: 'conv_0__11',
 12: 'conv_0__12',
 13: 'conv_0__13',
 14: 'conv_0__14',
 15: 'conv_0__15',
 16: 'conv_0__16',
 17: 'conv_0__17',
 18: 'conv_0__18',
 19: 'conv_0__19',
 20: 'conv_0__20',
 21: 'conv_0__21',
 22: 'conv_0__22',
 23: 'conv_0__23',
 24: 'conv_0__24',
 25: 'conv_0__25',
 26: 'conv_0__26',
 27: 'conv_0__27',
 28: 'conv_0__28',
 29: 'conv_0__29',
 30: 'conv_0__30',
 31: 'conv_0__31',
 32: 'conv_0__32',
 33: 'conv_0__33',
 34: 'conv_0__34',
 35: 'conv_0__35',
 36: 'conv_0__36',
 37: 'conv_0__37',
 38: 'conv_0__38',
 39: 'conv_0__39',
 40: 'conv_0__40',
 41: 'conv_0__41',
 42: 'conv_0__42',
 43: 'conv_0__43',
 44: 'conv_0__44',
 45: 'conv_0__45',
 46: 'conv_0__46',
 47: 'conv_0__47',
 48: 'conv_0__48',
 49: 'conv_0__49',
 50: 'conv_0__50',
 51: 'conv_0__51',
 52: 'conv_0__52',
 53: 'conv_0_

In [97]:
jupyter_utils.unregister_always_reload()

In [96]:
wave

array([-1.50476340e-02,  6.97858781e-02,  8.84800702e-02,  9.12731141e-03,
       -5.25312163e-02,  3.28979194e-02,  2.42064431e-01,  1.82948083e-01,
       -1.39684632e-01, -4.82958913e-01, -1.23379983e-01,  6.80880342e-03,
       -3.81344445e-02,  1.96072981e-01,  3.10818404e-01,  4.09034818e-01,
        1.64875790e-01, -1.18460327e-01, -4.66735894e-03,  2.03558221e-01,
        3.16960186e-01,  5.17192900e-01,  7.90320516e-01,  7.88563251e-01,
        1.10666947e-02, -3.33487034e-01,  2.05729678e-01,  6.57138407e-01,
        2.90525794e-01,  1.32160690e-02,  4.79250401e-01,  4.84555870e-01,
       -2.55390018e-01, -6.53196871e-01,  1.76233009e-01,  7.11781979e-01,
        1.20576680e-01, -2.83224434e-01,  1.89226180e-01,  4.85013813e-01,
       -3.21361810e-01, -1.38050449e+00, -3.44842166e-01,  5.11183918e-01,
       -1.11791387e-01, -7.27834880e-01, -6.88717484e-01,  2.30010394e-02,
        2.43185803e-01, -1.89610028e+00, -6.13028586e-01,  3.64549398e-01,
       -1.47758037e-01, -

In [90]:
output_dict['activations']['conv_1'].shape

(27, 512)

In [91]:
output_dict['activations']['lstm'].shape

(27, 512)

In [5]:
X_test = np.concatenate(list(output_dict['activations'].values()), axis=1)

In [2]:
# # data_dict_train = np.load(os.path.join(log_dir, 'data', 'train_data.npz'), allow_pickle=True)
# # X_train = data_dict_train['X']
# # y_train = data_dict_train['y']

# data_dict_test = np.load(os.path.join(log_dir, 'data', 'test_data.npz'), allow_pickle=True)
# X_test = data_dict_test['X']
# y_test = data_dict_test['y']

# # ignore zero duration
# # X_train = X_train[y_train > 0]
# # y_train = y_train[y_train > 0]
# X_test = X_test[y_test > 0]
# y_test = y_test[y_test > 0]

# # we predict log duration
# # y_train = np.log(y_train)
# y_test = np.log(y_test)

In [6]:
# load pre-trained probe
probe = linear_probe.LinearProbe(X_test.shape[1], 1)
probe.load_state_dict(
    torch.load(os.path.join(log_dir, 'models', 'probe.pth')))

<All keys matched successfully>

In [7]:
# load pre-trained probe
probe = linear_probe.LinearProbe(512, 1)
probe.load_state_dict(
    torch.load(os.path.join(log_dir, 'models', 'probe_layer_3.pth')))

<All keys matched successfully>

In [125]:
X_test.shape

(27, 2048)

In [200]:
init_v

tensor([1.2197, 1.5519, 1.3159, 1.8684, 1.9971, 1.7245, 1.6203, 1.3540, 1.5621,
        1.7738, 1.3134, 1.9101, 2.3360, 2.0148, 1.5448, 1.4171, 1.8952, 1.5703,
        1.4389, 1.9447, 1.4678, 2.3672, 2.4775, 1.1877, 2.4823, 2.5334, 3.0877],
       grad_fn=<SqueezeBackward0>)

In [202]:
duration

tensor([2.1833, 2.5155, 2.2795, 2.8320, 2.9607, 2.6881, 2.5839, 2.3176, 2.5257,
        2.7374, 2.2770, 2.8737, 3.2996, 2.9784, 2.5084, 2.3807, 2.8588, 2.5339,
        2.4025, 2.9083, 2.4314, 3.3308, 3.4411, 2.1513, 3.4459, 3.4970, 4.0513],
       grad_fn=<SqueezeBackward0>)

In [40]:
X_test = np.concatenate(list(output_dict['activations'].values()), axis=1)

# load pre-trained probe
probe = linear_probe.LinearProbe(X_test.shape[1], 1)
probe.load_state_dict(
    torch.load(os.path.join(log_dir, 'models', 'probe.pth')))

# load pre-trained probe
def get_neuron_ordering_for_linear_regression(probe, search_stride=100):
    # class_idx = class_to_idx[class_name]
    weights = list(probe.parameters())[0].data.cpu().numpy()
    abs_weights = np.abs(weights[0])

    neuron_orderings = []
    for p in range(1, search_stride + 1):
        percentage = p / search_stride
        total_mass = np.sum(abs_weights)
        sorted_idx = np.argsort(abs_weights)[::-1]  # Sort in descending order
        cum_sums = np.cumsum(abs_weights[sorted_idx])
        selected_neurons = sorted_idx[cum_sums <= total_mass * percentage]
        neuron_orderings.extend(selected_neurons)

    # Remove duplicates while preserving order
    neuron_ordering = list(dict.fromkeys(neuron_orderings))

    return np.array(neuron_ordering)

ordering = get_neuron_ordering_for_linear_regression(probe)

n = 11
X_selected_n_test = ablation.filter_activations_keep_neurons(X_test, ordering[:n])

probe_selected_n = linear_probe.LinearProbe(X_selected_n_test.shape[1], 1)
probe_selected_n.load_state_dict(
    torch.load(os.path.join(log_dir, 'models', 'probe_selected_11.pth')))


# idx = 0

# X_test = np.concatenate(list(output_dict['activations'].values()), axis=1)

factor = 2
scale = 0.001
# x = torch.tensor(X_test[idx][None], dtype=torch.float32)
# layer_k_X_test = ablation.filter_activations_by_layers(X_test, [3], 4)
x = torch.tensor(X_selected_n_test, dtype=torch.float32)
# x = torch.tensor(X_test, dtype=torch.float32)


batch_size = x.shape[0]  # x의 배치 크기
init_v = probe_selected_n(x).squeeze()

# 목표 달성 여부를 추적하는 마스크 초기화 (모두 False로 시작)
achieved_goal = torch.zeros(batch_size, dtype=torch.bool)

for _ in range(1000):
    with torch.enable_grad():
        x.requires_grad_()

        duration = probe_selected_n(x).squeeze()
        print(duration)
           
        # # 목표 달성 여부 업데이트
        # if factor > 1:
        #     achieved_goal |= (duration > init_v + np.log(factor))
        # else:
        #     achieved_goal |= (duration < init_v + np.log(factor))

        grad = torch.autograd.grad(
            [duration.sum()],  # 배치에 대한 총합을 사용
            [x]
        )[0]

        x = x.detach()
        # 목표에 도달하지 않은 샘플만 업데이트
        if factor > 1:
            x[~achieved_goal] = x[~achieved_goal] + scale * grad[~achieved_goal]
        else:
            x[~achieved_goal] = x[~achieved_goal] - scale * grad[~achieved_goal]

        # # 목표 달성 여부 업데이트
        # if factor > 1:
        #     achieved_goal |= (duration > init_v + np.log(factor))
        # else:
        #     achieved_goal |= (duration < init_v + np.log(factor))
            
        # # 모든 샘플이 목표에 도달했으면 루프 종료
        # if achieved_goal.all():
        #     break


tensor([1.1787, 1.3523, 1.8624, 1.5678, 2.0211, 1.7013, 1.6669, 1.8060, 1.4949,
        1.6205, 1.3156, 1.9687, 2.4035, 1.7652, 2.0768, 1.5477, 1.9867, 1.7562,
        1.4923, 2.0964, 2.2788, 1.8179, 2.4933, 1.7935, 2.3931, 2.5958, 3.4143],
       grad_fn=<SqueezeBackward0>)
tensor([1.1868, 1.3605, 1.8706, 1.5759, 2.0293, 1.7095, 1.6750, 1.8142, 1.5030,
        1.6287, 1.3237, 1.9768, 2.4116, 1.7733, 2.0849, 1.5558, 1.9948, 1.7643,
        1.5004, 2.1046, 2.2869, 1.8261, 2.5014, 1.8016, 2.4012, 2.6039, 3.4224],
       grad_fn=<SqueezeBackward0>)
tensor([1.1950, 1.3686, 1.8787, 1.5841, 2.0374, 1.7176, 1.6832, 1.8223, 1.5111,
        1.6368, 1.3318, 1.9850, 2.4197, 1.7815, 2.0931, 1.5640, 2.0030, 1.7724,
        1.5085, 2.1127, 2.2950, 1.8342, 2.5095, 1.8097, 2.4094, 2.6120, 3.4306],
       grad_fn=<SqueezeBackward0>)
tensor([1.2031, 1.3767, 1.8868, 1.5922, 2.0455, 1.7257, 1.6913, 1.8304, 1.5193,
        1.6449, 1.3400, 1.9931, 2.4278, 1.7896, 2.1012, 1.5721, 2.0111, 1.7806,
        1.51

In [42]:
def get_neuronidx2name(d):
    mapping = {}
    current_start_index = 0
    for layer_name, activations in d.items():
        # Calculate the end index for this activation
        end_index = current_start_index + activations.shape[1] - 1
        for i in range(current_start_index, end_index + 1):
            mapping[i] = f'{layer_name}__{i - current_start_index}'
        current_start_index = end_index + 1
    return mapping

nid2name = get_neuronidx2name(output_dict['activations'])

new_dict = output_dict['activations'].copy()

for oi, o in enumerate(ordering[:n]):
    name, i = nid2name[o].split('__')
    new_dict[name][:, int(i)] = x[:, oi].cpu().detach().numpy()
    print(name, i)

for k, v in new_dict.items():
    new_dict[k] = torch.tensor(v).cuda()[None, :] 

text2mel.set_modified_activations(new_dict)


tts = TTSInterface(
    device=device,
    text2mel=text2mel,
    mel2wave=mel2wave 
).to(device)

text = text_dict['LJ001-0002']
output_dict = tts.forward(text)
wave = output_dict['wave']

ipd.Audio(wave, rate=tts.sampling_rate)

conv_2 380
lstm 447
lstm 494
lstm 39
lstm 328
conv_2 78
lstm 197
conv_2 444
lstm 323
lstm 26
lstm 165
(80, 158)


In [15]:
# idx = 0

X_test = np.concatenate(list(output_dict['activations'].values()), axis=1)

factor = 1.5
scale = 0.001
# x = torch.tensor(X_test[idx][None], dtype=torch.float32)
layer_k_X_test = ablation.filter_activations_by_layers(X_test, [3], 4)
x = torch.tensor(layer_k_X_test, dtype=torch.float32)
# x = torch.tensor(X_test, dtype=torch.float32)


batch_size = x.shape[0]  # x의 배치 크기
init_v = probe(x).squeeze()

# 목표 달성 여부를 추적하는 마스크 초기화 (모두 False로 시작)
achieved_goal = torch.zeros(batch_size, dtype=torch.bool)

for _ in range(1000):
    with torch.enable_grad():
        x.requires_grad_()

        duration = probe(x).squeeze()
        print(duration)
           
        # # 목표 달성 여부 업데이트
        # if factor > 1:
        #     achieved_goal |= (duration > init_v + np.log(factor))
        # else:
        #     achieved_goal |= (duration < init_v + np.log(factor))

        grad = torch.autograd.grad(
            [duration.sum()],  # 배치에 대한 총합을 사용
            [x]
        )[0]

        x = x.detach()
        # 목표에 도달하지 않은 샘플만 업데이트
        if factor > 1:
            x[~achieved_goal] = x[~achieved_goal] + scale * grad[~achieved_goal]
        else:
            x[~achieved_goal] = x[~achieved_goal] - scale * grad[~achieved_goal]

        # 목표 달성 여부 업데이트
        if factor > 1:
            achieved_goal |= (duration > init_v + np.log(factor))
        else:
            achieved_goal |= (duration < init_v + np.log(factor))
            
        # 모든 샘플이 목표에 도달했으면 루프 종료
        if achieved_goal.all():
            break

# print(x[0])

    # if duration 
    # print(torch.exp(probe(x)))

tensor([1.0320, 1.5649, 1.1399, 1.5950, 1.9607, 1.7930, 1.6152, 1.1400, 1.4732,
        1.6540, 1.1687, 1.6740, 2.2843, 2.2464, 1.5160, 1.3369, 1.7331, 1.4971,
        1.3304, 1.9861, 1.2404, 2.1410, 2.5059, 1.2847, 2.5118, 2.3610, 2.5927],
       grad_fn=<SqueezeBackward0>)
tensor([1.0333, 1.5662, 1.1412, 1.5963, 1.9620, 1.7943, 1.6165, 1.1413, 1.4745,
        1.6553, 1.1700, 1.6753, 2.2857, 2.2477, 1.5173, 1.3382, 1.7344, 1.4984,
        1.3318, 1.9874, 1.2417, 2.1423, 2.5073, 1.2860, 2.5131, 2.3623, 2.5940],
       grad_fn=<SqueezeBackward0>)
tensor([1.0346, 1.5675, 1.1425, 1.5977, 1.9634, 1.7957, 1.6178, 1.1427, 1.4759,
        1.6566, 1.1713, 1.6766, 2.2870, 2.2491, 1.5187, 1.3396, 1.7357, 1.4997,
        1.3331, 1.9888, 1.2431, 2.1436, 2.5086, 1.2873, 2.5144, 2.3636, 2.5954],
       grad_fn=<SqueezeBackward0>)
tensor([1.0359, 1.5688, 1.1438, 1.5990, 1.9647, 1.7970, 1.6192, 1.1440, 1.4772,
        1.6579, 1.1726, 1.6779, 2.2883, 2.2504, 1.5200, 1.3409, 1.7370, 1.5010,
        1.33

In [17]:
x

tensor([[ 0.3478, -0.4265, -0.0270,  ...,  0.4323, -0.0296,  0.1919],
        [ 0.2458, -0.7717, -0.0217,  ...,  0.3084, -0.0309,  0.2093],
        [ 0.0915, -0.9127, -0.9017,  ...,  0.3171,  0.2418, -0.1866],
        ...,
        [ 0.2973, -0.1370, -0.0384,  ...,  0.0136, -0.0313,  0.2604],
        [ 0.2170, -0.0843, -0.0205,  ...,  0.4892, -0.0294,  0.1255],
        [ 0.1708, -0.1677, -0.9792,  ...,  0.0642,  0.5372, -0.0457]])

In [16]:
modified_activations = {
    # 'lstm': torch.from_numpy(activations2dict(new_activations)['lstm'][None, :]).cuda(),
    'lstm': x[None, :].cuda()
}

text2mel.set_modified_activations(modified_activations)


tts = TTSInterface(
    device=device,
    text2mel=text2mel,
    mel2wave=mel2wave 
).to(device)

text = text_dict['LJ001-0002']
output_dict = tts.forward(text)
wave = output_dict['wave']

ipd.Audio(wave, rate=tts.sampling_rate)

(80, 164)


/nas/users/dahye/kw/tts/ttsxai/src/tacotron2/audio_processing.py:50: FutureWarning: Pass size=1024 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  win_sq = librosa_util.pad_center(win_sq, n_fft)


In [277]:
idx = 26
factor = 5
scale = 0.1
# x = torch.tensor(X_test[idx][None], dtype=torch.float32)
layer_k_X_test = ablation.filter_activations_by_layers(X_test, [3], 4)
x = torch.tensor(layer_k_X_test[idx][None], dtype=torch.float32)
# x = torch.tensor(X, dtype=torch.float32)
init_v = probe(x).squeeze()

for _ in range(100):
    with torch.enable_grad():
        x.requires_grad_()

        duration = probe(x).squeeze()
           
        # print(duration)
        grad = torch.autograd.grad(
            [duration], 
            [x]
        )[0]

        x = x.detach()
    if factor > 1:
        x = x + scale * grad
        if duration > init_v + np.log(factor):
            break
    else:
        x = x - scale * grad
        if duration < init_v + np.log(factor):
            break

    # if duration 
    # print(torch.exp(probe(x)))
print(x)

tensor([[ 5.9710e-02, -1.8150e-01, -1.0478e+00,  3.7657e-02,  7.4398e-02,
         -3.4559e-01,  2.4320e-03, -8.5731e-01, -9.8279e-01, -7.5615e-02,
          7.6526e-02,  8.7060e-01, -1.0376e+00, -1.0984e-01,  2.3614e-01,
         -4.1729e-02,  3.4809e-01, -1.6415e-01,  1.7070e-01, -1.0532e+00,
          2.9047e-02, -6.5692e-01, -4.2080e-02,  6.2479e-01,  1.0977e+00,
         -2.9739e-02,  3.7654e-01, -1.0446e+00,  3.6889e-01,  1.1141e+00,
         -1.0091e-02, -8.6400e-01, -2.3787e-01, -1.9427e-01, -3.7499e-01,
         -7.2823e-01, -5.5742e-01, -1.6886e-02,  2.9766e-03,  1.3727e+00,
         -1.4366e-02,  4.9350e-01, -9.9375e-01,  7.4916e-02,  1.1523e-02,
          9.3994e-01, -1.9504e-01, -7.8862e-02,  2.2436e-02, -9.3814e-02,
         -3.9502e-02,  1.8522e-02,  6.5666e-01,  3.8785e-02, -2.0656e-01,
          3.3804e-01,  1.2332e-02, -3.5601e-01,  5.1967e-01, -7.2439e-01,
          4.4487e-02, -6.0636e-01,  7.3740e-02,  4.0274e-01, -3.3029e-01,
         -1.1138e+00,  1.8399e-01, -1.

In [3]:
# load pre-trained probe
probe = linear_probe.LinearProbe(X_train.shape[1], 1)
probe.load_state_dict(
    torch.load(os.path.join(log_dir, 'models', 'probe.pth')))

NameError: name 'X_train' is not defined

In [6]:
import torch.optim as optim

# PyTorch의 Tensor로 변환
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)

In [7]:

# 모델 및 최적화 설정
probe.eval()  # 평가 모드
optimizer = optim.SGD([X_train_tensor.requires_grad_()], lr=0.01)

# 특정 샘플 선택
sample_index = 0  # 예시로 첫 번째 샘플 선택
target_log_duration = y_train_tensor[sample_index] * 2  # 목표 log-duration

# 손실 함수 정의
criterion = nn.MSELoss()

# 업데이트 횟수
num_updates = 100

In [9]:
probe.cuda()

LinearProbe(
  (linear): Linear(in_features=2048, out_features=1, bias=True)
)

In [13]:
loss

tensor(1.0891, device='cuda:0', grad_fn=<MseLossBackward0>)

In [12]:
# 입력을 수정하기 위한 반복
for i in range(num_updates):
    print(i)
    optimizer.zero_grad()  # 기울기 초기화
    current_prediction = probe(X_train_tensor[sample_index].unsqueeze(0).cuda())
    loss = criterion(current_prediction, target_log_duration.unsqueeze(0).cuda())
    loss.backward()  # 역전파
    optimizer.step()  # 입력 업데이트

# 업데이트된 입력을 저장
modified_input = X_train_tensor[sample_index].detach().numpy()

# 결과 확인
print("Original Input:", X_train[sample_index])
print("Modified Input:", modified_input)

0


/home/dahye/anaconda3/envs/ttsxai/lib/python3.7/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


1
2
3
4
5
6
7


KeyboardInterrupt: 

In [17]:
import torch

# PyTorch의 Tensor로 변환
X_train_tensor = torch.tensor(X_train, dtype=torch.float32, requires_grad=True)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)

# 모델 평가 모드
probe.eval()

# 특정 샘플 선택
sample_index = 0  # 예시로 첫 번째 샘플 선택
target_log_duration = y_train_tensor[sample_index] * 2  # 목표 log-duration

# 손실 함수 정의
criterion = nn.MSELoss()

# 업데이트 횟수
num_updates = 100

# 입력을 수정하기 위한 반복
for i in range(num_updates):
    print(i)
    # 현재 예측 계산
    current_prediction = probe(X_train_tensor[sample_index].unsqueeze(0).cuda())

    # 손실 계산
    loss = criterion(current_prediction, target_log_duration.unsqueeze(0).cuda())

    # 기울기 계산
    gradients = torch.autograd.grad(outputs=loss, inputs=X_train_tensor[sample_index])

    # 입력 수정
    X_train_tensor[sample_index] -= 0.01 * gradients[0]  # 학습률 0.01

# 업데이트된 입력을 저장
modified_input = X_train_tensor[sample_index].detach().numpy()

# 결과 확인
print("Original Input:", X_train[sample_index])
print("Modified Input:", modified_input)

0


/home/dahye/anaconda3/envs/ttsxai/lib/python3.7/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


RuntimeError: One of the differentiated Tensors appears to not have been used in the graph. Set allow_unused=True if this is the desired behavior.

In [5]:

# 모델 및 최적화 설정
probe.eval()  # 평가 모드
optimizer = optim.SGD([X_train_tensor.requires_grad_()], lr=0.01)

# 특정 샘플 선택
sample_index = 0  # 예시로 첫 번째 샘플 선택
target_log_duration = y_train_tensor[sample_index] * 2  # 목표 log-duration

# 손실 함수 정의
criterion = nn.MSELoss()

# 업데이트 횟수
num_updates = 100

# 입력을 수정하기 위한 반복
for _ in range(num_updates):
    optimizer.zero_grad()  # 기울기 초기화
    current_prediction = probe(X_train_tensor[sample_index].unsqueeze(0))
    loss = criterion(current_prediction, target_log_duration.unsqueeze(0))
    loss.backward()  # 역전파
    optimizer.step()  # 입력 업데이트

# 업데이트된 입력을 저장
modified_input = X_train_tensor[sample_index].detach().numpy()

# 결과 확인
print("Original Input:", X_train[sample_index])
print("Modified Input:", modified_input)

/home/dahye/anaconda3/envs/ttsxai/lib/python3.7/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


KeyboardInterrupt: 

# 나중에는 selected neurons로!

# 나중에는 특정 layer로 !